In [75]:
import os
import torch
import os.path as osp
import json, pickle
import numpy as np
import pandas as pd
import networkx as nx

from utils_copy import *
from tqdm import tqdm
from rdkit import Chem
from rdkit import RDConfig
from collections import OrderedDict
from rdkit.Chem import ChemicalFeatures
fdef_name = osp.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
chem_feature_factory = ChemicalFeatures.BuildFeatureFactory(fdef_name)

In [76]:
VOCAB_PROTEIN = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y',
                'X']

def read_data(filename):
    df = pd.read_csv('dataset/'+ filename)
    drugs, prots = list(df['compound_iso_smiles']),list(df['target_sequence'])
    return drugs, prots


def dic_normalize(dic):
    max_value = dic[max(dic, key=dic.get)]
    min_value = dic[min(dic, key=dic.get)]
    # print(max_value)
    interval = float(max_value) - float(min_value)
    for key in dic.keys():
        dic[key] = (dic[key] - min_value) / interval
    dic['X'] = (max_value + min_value) / 2.0
    return dic

seq_dict = {v:(i+1) for i,v in enumerate(VOCAB_PROTEIN)}
pro_res_aliphatic_table = ['A', 'I', 'L', 'M', 'V']
pro_res_aromatic_table = ['F', 'W', 'Y']
pro_res_polar_neutral_table = ['C', 'N', 'Q', 'S', 'T']
pro_res_acidic_charged_table = ['D', 'E']
pro_res_basic_charged_table = ['H', 'K', 'R']

res_weight_table = {'A': 71.08, 'C': 103.15, 'D': 115.09, 'E': 129.12, 'F': 147.18, 'G': 57.05, 'H': 137.14,
                    'I': 113.16, 'K': 128.18, 'L': 113.16, 'M': 131.20, 'N': 114.11, 'P': 97.12, 'Q': 128.13,
                    'R': 156.19, 'S': 87.08, 'T': 101.11, 'V': 99.13, 'W': 186.22, 'Y': 163.18}
res_weight_table['X'] = np.average([res_weight_table[k] for k in res_weight_table.keys()])

res_pka_table = {'A': 2.34, 'C': 1.96, 'D': 1.88, 'E': 2.19, 'F': 1.83, 'G': 2.34, 'H': 1.82, 'I': 2.36,
                 'K': 2.18, 'L': 2.36, 'M': 2.28, 'N': 2.02, 'P': 1.99, 'Q': 2.17, 'R': 2.17, 'S': 2.21,
                 'T': 2.09, 'V': 2.32, 'W': 2.83, 'Y': 2.32}
res_pka_table['X'] = np.average([res_pka_table[k] for k in res_pka_table.keys()])

res_pkb_table = {'A': 9.69, 'C': 10.28, 'D': 9.60, 'E': 9.67, 'F': 9.13, 'G': 9.60, 'H': 9.17,
                 'I': 9.60, 'K': 8.95, 'L': 9.60, 'M': 9.21, 'N': 8.80, 'P': 10.60, 'Q': 9.13,
                 'R': 9.04, 'S': 9.15, 'T': 9.10, 'V': 9.62, 'W': 9.39, 'Y': 9.62}
res_pkb_table['X'] = np.average([res_pkb_table[k] for k in res_pkb_table.keys()])

res_pkx_table = {'A': 0.00, 'C': 8.18, 'D': 3.65, 'E': 4.25, 'F': 0.00, 'G': 0, 'H': 6.00,
                 'I': 0.00, 'K': 10.53, 'L': 0.00, 'M': 0.00, 'N': 0.00, 'P': 0.00, 'Q': 0.00,
                 'R': 12.48, 'S': 0.00, 'T': 0.00, 'V': 0.00, 'W': 0.00, 'Y': 0.00}
res_pkx_table['X'] = np.average([res_pkx_table[k] for k in res_pkx_table.keys()])

res_pl_table = {'A': 6.00, 'C': 5.07, 'D': 2.77, 'E': 3.22, 'F': 5.48, 'G': 5.97, 'H': 7.59,
                'I': 6.02, 'K': 9.74, 'L': 5.98, 'M': 5.74, 'N': 5.41, 'P': 6.30, 'Q': 5.65,
                'R': 10.76, 'S': 5.68, 'T': 5.60, 'V': 5.96, 'W': 5.89, 'Y': 5.96}
res_pl_table['X'] = np.average([res_pl_table[k] for k in res_pl_table.keys()])

res_hydrophobic_ph2_table = {'A': 47, 'C': 52, 'D': -18, 'E': 8, 'F': 92, 'G': 0, 'H': -42, 'I': 100,
                             'K': -37, 'L': 100, 'M': 74, 'N': -41, 'P': -46, 'Q': -18, 'R': -26, 'S': -7,
                             'T': 13, 'V': 79, 'W': 84, 'Y': 49}
res_hydrophobic_ph2_table['X'] = np.average([res_hydrophobic_ph2_table[k] for k in res_hydrophobic_ph2_table.keys()])

res_hydrophobic_ph7_table = {'A': 41, 'C': 49, 'D': -55, 'E': -31, 'F': 100, 'G': 0, 'H': 8, 'I': 99,
                             'K': -23, 'L': 97, 'M': 74, 'N': -28, 'P': -46, 'Q': -10, 'R': -14, 'S': -5,
                             'T': 13, 'V': 76, 'W': 97, 'Y': 63}
res_hydrophobic_ph7_table['X'] = np.average([res_hydrophobic_ph7_table[k] for k in res_hydrophobic_ph7_table.keys()])

# nomarlize the residue feature
res_weight_table = dic_normalize(res_weight_table)
res_pka_table = dic_normalize(res_pka_table)
res_pkb_table = dic_normalize(res_pkb_table)
res_pkx_table = dic_normalize(res_pkx_table)
res_pl_table = dic_normalize(res_pl_table)
res_hydrophobic_ph2_table = dic_normalize(res_hydrophobic_ph2_table)
res_hydrophobic_ph7_table = dic_normalize(res_hydrophobic_ph7_table)

def one_hot_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception('input {0} not in allowable set{1}:'.format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

# one ont encoding with unknown symbol
def one_hot_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

def encoding_unk(x, allowable_set):
    list = [False for i in range(len(allowable_set))]
    i = 0
    for atom in x:
        if atom in allowable_set:
            list[allowable_set.index(atom)] = True
            i += 1
    if i != len(x):
        list[-1] = True
    return list

def residue_features(residue):
    res_property1 = [1 if residue in pro_res_aliphatic_table else 0, 1 if residue in pro_res_aromatic_table else 0,
                     1 if residue in pro_res_polar_neutral_table else 0,
                     1 if residue in pro_res_acidic_charged_table else 0,
                     1 if residue in pro_res_basic_charged_table else 0]
    res_property2 = [res_weight_table[residue], res_pka_table[residue], res_pkb_table[residue], res_pkx_table[residue],
                     res_pl_table[residue], res_hydrophobic_ph2_table[residue], res_hydrophobic_ph7_table[residue]]

    return np.array(res_property1 + res_property2)

# target feature for target graph
def PSSM_calculation(aln_file, pro_seq):
    pfm_mat = np.zeros((len(VOCAB_PROTEIN), len(pro_seq)))
    with open(aln_file, 'r') as f:
        line_count = len(f.readlines())
        for line in f.readlines():
            if len(line) != len(pro_seq):
                print('error', len(line), len(pro_seq))
                continue
            count = 0
            for res in line:
                if res not in VOCAB_PROTEIN:
                    count += 1
                    continue
                pfm_mat[VOCAB_PROTEIN.index(res), count] += 1
                count += 1

    pseudocount = 0.8
    ppm_mat = (pfm_mat + pseudocount / 4) / (float(line_count) + pseudocount)
    pssm_mat = ppm_mat

    return pssm_mat

def seq_feature(pro_seq):
    pro_hot = np.zeros((len(pro_seq), len(VOCAB_PROTEIN)))
    pro_property = np.zeros((len(pro_seq), 12))
    for i in range(len(pro_seq)):
        # if 'X' in pro_seq:
        #     print(pro_seq)
        pro_hot[i,] = one_hot_encoding(pro_seq[i], VOCAB_PROTEIN)
        pro_property[i,] = residue_features(pro_seq[i])
    return np.concatenate((pro_hot, pro_property), axis=1)

def target_feature(aln_file, pro_seq):
    pssm = PSSM_calculation(aln_file, pro_seq)
    other_feature = seq_feature(pro_seq)

    return np.concatenate((np.transpose(pssm, (1, 0)), other_feature), axis=1)

def target2feature(target_key, target_sequence, aln_dir):
    aln_file = os.path.join(aln_dir, target_key + '.aln')
    feature = target_feature(aln_file, target_sequence)

    return feature


# target sequence to target graph
def sequence2graph(target_key, target_sequence, distance_dir):
    target_edge_index = []
    target_edge_distance = []
    target_size = len(target_sequence)

    target_feature = seq_feature(target_sequence)
    contact_map_file = os.path.join(distance_dir, target_key + '.npy')
    distance_map = np.load(contact_map_file)

    for i in range(target_size):
        distance_map[i, i] = 1
        if i + 1 < target_size:
            distance_map[i, i + 1] = 1
    index_row, index_col = np.where(distance_map >= 0.5)  # for threshold

    for i, j in zip(index_row, index_col):
        target_edge_index.append([i, j])  # dege
        target_edge_distance.append(distance_map[i, j])  # edge weight

    target_feature = torch.Tensor(target_feature)
    target_edge_index = torch.LongTensor(target_edge_index).transpose(1, 0)
    target_edge_distance = torch.FloatTensor(target_edge_distance)

    return target_size, target_feature, target_edge_index, target_edge_distance

def get_nodes(g):
    feat = []
    for n, d in g.nodes(data=True):
        h_t = []
        h_t += [int(d['a_type'] == x) for x in ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na',
                                                'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb',
                                                'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H', 'Li',
                                                'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
                                                'Pt', 'Hg', 'Pb', 'X']]
        h_t.append(d['a_num'])
        h_t.append(d['acceptor'])
        h_t.append(d['donor'])
        h_t.append(int(d['aromatic']))
        h_t += [int(d['degree'] == x) for x in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
        h_t += [int(d['ImplicitValence'] == x) for x in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
        h_t += [int(d['num_h'] == x) for x in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]
        h_t += [int(d['hybridization'] == x) for x in (Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2, Chem.rdchem.HybridizationType.SP3)]
        # 5 more
        h_t.append(d['ExplicitValence'])
        h_t.append(d['FormalCharge'])
        h_t.append(d['NumExplicitHs'])
        h_t.append(d['NumRadicalElectrons'])
        feat.append((n, h_t))
    feat.sort(key=lambda item: item[0])
    node_attr = torch.FloatTensor([item[1] for item in feat])
    return node_attr

def get_edges(g):
    e = {}
    for n1, n2, d in g.edges(data=True):
        e_t = [int(d['b_type'] == x)
                for x in (Chem.rdchem.BondType.SINGLE, \
                            Chem.rdchem.BondType.DOUBLE, \
                            Chem.rdchem.BondType.TRIPLE, \
                            Chem.rdchem.BondType.AROMATIC)]

        e_t.append(int(d['IsConjugated'] == False))
        e_t.append(int(d['IsConjugated'] == True))
        e[(n1, n2)] = e_t

    edge_index = torch.LongTensor(list(e.keys())).transpose(0, 1)
    edge_attr = torch.FloatTensor(list(e.values()))


    return edge_index, edge_attr

# mol smile to mol graph edge index
def smile2graph(smile):
    mol = Chem.MolFromSmiles(smile)

    feats = chem_feature_factory.GetFeaturesForMol(mol)
    mol_size = mol.GetNumAtoms()
    g = nx.DiGraph()
    
    # Create nodes
    for i in range(mol.GetNumAtoms()):
        atom_i = mol.GetAtomWithIdx(i)
        g.add_node(i,
                a_type=atom_i.GetSymbol(),
                a_num=atom_i.GetAtomicNum(),
                acceptor=0,
                donor=0,
                aromatic=atom_i.GetIsAromatic(),
                hybridization=atom_i.GetHybridization(),
                num_h=atom_i.GetTotalNumHs(),
                degree = atom_i.GetDegree(),
                # 5 more node features
                ExplicitValence=atom_i.GetExplicitValence(),
                FormalCharge=atom_i.GetFormalCharge(),
                ImplicitValence=atom_i.GetImplicitValence(),
                NumExplicitHs=atom_i.GetNumExplicitHs(),
                NumRadicalElectrons=atom_i.GetNumRadicalElectrons(),
            )
            
    for i in range(len(feats)):
        if feats[i].GetFamily() == 'Donor':
            node_list = feats[i].GetAtomIds()
            for n in node_list:
                g.nodes[n]['donor'] = 1
        elif feats[i].GetFamily() == 'Acceptor':
            node_list = feats[i].GetAtomIds()
            for n in node_list:
                g.nodes[n]['acceptor']
    # Read Edges
    for i in range(mol.GetNumAtoms()):
        for j in range(mol.GetNumAtoms()):
            e_ij = mol.GetBondBetweenAtoms(i, j)
            if e_ij is not None:
                g.add_edge(i, j,
                            b_type=e_ij.GetBondType(),
                            # 1 more edge features 2 dim
                            IsConjugated=int(e_ij.GetIsConjugated()),
                            )
                
    node_attr = get_nodes(g)
    edge_index, edge_attr = get_edges(g)         

    return mol_size, node_attr, edge_index, edge_attr

def create_dataset(dataset):
    dataset_dir = os.path.join('dataset',dataset)
    # drug smiles
    ligands = json.load(open(os.path.join(dataset_dir, 'ligands_can.txt')), object_pairs_hook=OrderedDict)
    # protein sequences
    proteins = json.load(open(os.path.join(dataset_dir, 'proteins.txt')), object_pairs_hook=OrderedDict)

    # load protein feature and predicted distance map
    process_dir = os.path.join('dataset/')
    pro_distance_dir = os.path.join(process_dir, dataset, 'pconsc4')  # numpy .npy file
    
    # dataset process
    drugs = []  # rdkit entity
    prots = []  # sequences
    prot_keys = []  # protein id (or name)
    drug_smiles = []  # smiles
    # create molecule graph
    print("create molecule graph ...")
    # smiles
    for d in ligands.keys():
        lg = ligands[d]
        drugs.append(lg)
        drug_smiles.append(ligands[d])
        smile_graph = {}

    for i in tqdm(range(len(drugs))):
        smile = drugs[i]

        g_d = smile2graph(smile)
        smile_graph[smile] = g_d

    print("create protein graph ...")  
    # seqs
    for t in proteins.keys():
        prots.append(proteins[t])
        prot_keys.append(t)  

    target_graph = {}
    for i in tqdm(range(len(prot_keys))):
        key = prot_keys[i]
        protein = prots[i]
        g_t = sequence2graph(key, protein, pro_distance_dir)
        target_graph[protein] = g_t
    
    # read files(train and test)
    test_csv = dataset + '/raw/test.csv'
    test_drugs, test_prots = read_data(test_csv)
    test_drugs, test_prots = np.asarray(test_drugs), np.asarray(test_prots)

    test_data = DTADataset(root='dataset', dataset=dataset + '_' + 'test', drug_smiles=test_drugs, target_sequence=test_prots,smile_graph=smile_graph, target_graph=target_graph)
    return test_data

In [77]:
VOCAB_PROTEIN = { "A": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, 
				"H": 7, "I": 8, "K": 9, "L": 10, "M": 11, "N": 12, 
				"P": 13, "Q": 14, "R": 15, "S": 16, "T": 17, "V": 18, 
				"W": 19, "Y": 20, "X": 21}

def PROTEIN2INT(target):
    return [VOCAB_PROTEIN[s] for s in target] 

# initialize the dataset

class DTADataset(InMemoryDataset):
    def __init__(self, root='data', dataset='davis',
                drug_smiles=None, target_sequence=None, x=None, x_mask=None, xt=None, xt_mask=None, y=None, transform=None,
                pre_transform=None, smile_graph=None,  target_graph=None):
        super(DTADataset, self).__init__(root, transform, pre_transform)
        self.dataset = dataset
        self.drug_smiles = drug_smiles
        self.target_sequence = target_sequence
        self.y = y
        self.smile_graph = smile_graph
        self.target_graph = target_graph

        self.process(drug_smiles, target_sequence, x, x_mask, xt, xt_mask, smile_graph, target_graph)

    @property
    def raw_file_names(self):
        pass

    @property
    def processed_file_names(self):
        return [self.dataset + '_data_mol.pt', self.dataset + '_data_pro.pt']

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def _download(self):
        pass

    def _process(self):
        if not os.path.exists(self.processed_dir):
            os.makedirs(self.processed_dir)

    def process(self, drug_smiles=None, target_sequence=None, x=None, x_mask=None, xt=None, xt_mask=None, smile_graph=None, target_graph=None):
        assert (len(drug_smiles) == len(target_sequence)), 'The three lists must have the same length!'
        data_list_mol = []
        data_list_pro = []

        data_len = len(drug_smiles)
        print('loading tensors ...')
        for i in tqdm(range(data_len)):

            smiles = drug_smiles[i]
            tar_seq = target_sequence[i]
            if x is not None:
                drug = x[i]
            if x_mask is not None:
                drug_mask = x_mask[i]
            if xt is not None:
                target = xt[i]
            if xt_mask is not None:
                target_mask = xt_mask[i]

            mol_size, mol_features, mol_edge_index, mol_edge_attr= smile_graph[smiles]
            target_size, target_features, target_edge_index, target_edge_weight = target_graph[tar_seq]
            
            GCNData_mol = DATA.Data(x=mol_features,
                                    edge_index=mol_edge_index,
                                    edge_attr = mol_edge_attr,)
            if x is not None:
                GCNData_mol.drug = torch.LongTensor([drug])
            if x_mask is not None:
                GCNData_mol.drug_mask = torch.LongTensor([drug_mask])
            GCNData_mol.__setitem__('c_size', torch.LongTensor([mol_size]))

            target_seq = PROTEIN2INT(tar_seq)
            target_seq_len = 1200
            if len(target_seq) < target_seq_len:
                pro_seq_emb = np.pad(target_seq, (0, target_seq_len- len(target_seq)))
            else:
                pro_seq_emb = target_seq[:target_seq_len]

            GCNData_pro = DATA.Data(x=target_features,
                                    edge_index=target_edge_index,
                                    edge_attr = target_edge_weight,
                                    pro_emb = torch.LongTensor([pro_seq_emb]),)
            
            if xt is not None:
                GCNData_pro.target = torch.LongTensor([target])
            if xt_mask is not None:
                GCNData_pro.target_mask = torch.LongTensor([target_mask])
            GCNData_pro.__setitem__('target_size', torch.LongTensor([target_size]))

            data_list_mol.append(GCNData_mol)
            data_list_pro.append(GCNData_pro)


        if self.pre_filter is not None:
            data_list_mol = [data for data in data_list_mol if self.pre_filter(data)]
            data_list_pro = [data for data in data_list_pro if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list_mol = [self.pre_transform(data) for data in data_list_mol]
            data_list_pro = [self.pre_transform(data) for data in data_list_pro]


        self.data_mol = data_list_mol
        self.data_pro = data_list_pro


    def __len__(self):
        return len(self.drug_smiles)

    def __getitem__(self, idx):
        # return GNNData_mol, GNNData_pro
        return self.data_mol[idx], self.data_pro[idx]


In [78]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from typing import Any, Dict, Optional
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
from torch_geometric.nn import GPSConv,GCNConv,GATConv,GINConv,global_max_pool as gmp, global_mean_pool as gep
from collections import OrderedDict
from torch_geometric.nn.resolver import normalization_resolver
from torch_geometric.utils import to_dense_batch

from torch.nn import (
    Linear,
    ModuleList,
    ReLU,
    Sequential,
    Dropout
)

class Attention(nn.Module):
    def __init__(self, in_size, hidden_size=64):
        super(Attention, self).__init__()
        self.project_x = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )
        self.project_xt = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )
        self.project_st = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1, bias=False)
        )
    def forward(self, x, xt,st):
        x = self.project_x(x)
        xt = self.project_xt(xt)
        st = self.project_st(st)
        a = torch.cat((x, xt, st), 1)
        a = torch.softmax(a, dim=1)
        return a

class GraphConv(nn.Module):
    def __init__(self, in_channel:int, local_conv: Optional[MessagePassing], heads: int, dropout: float):
        super().__init__()
        
        self.in_channel = in_channel
        self.local_conv = local_conv
        self.heads = heads
        self.dropout = dropout
        self.attn = nn.MultiheadAttention(in_channel, heads, batch_first= True)
        self.mlp = Sequential(
            Linear(in_channel, in_channel * 2),
            ReLU(),
            Dropout(dropout),
            Linear(in_channel * 2, in_channel),
            Dropout(dropout)
        )
        
        self.norm1 = normalization_resolver('batch_norm', in_channel)
        self.norm2 = normalization_resolver('batch_norm', in_channel)
        self.norm3 = normalization_resolver('batch_norm', in_channel)
        
    def forward(self, x: Tensor,  edge_index: Adj, batch: Optional[torch.Tensor] = None)->Tensor:
        
        hs = []
        h = self.local_conv(x,edge_index)
        h = F.dropout(h, p=self.dropout, training= self.training)
        h = h + x
        h = self.norm1(h)
        hs.append(h)
        
        # Global attention transformer-style model.
        h, mask = to_dense_batch(x, batch)
        h, _ = self.attn(h, h, h, key_padding_mask=~mask, need_weights=False)
        h = h[mask]
        h = F.dropout(h,p=self.dropout, training= self.training)
        h = h + x
        h = self.norm2(h)
        hs.append(h)
        
        # Combine local and global outputs.
        out = sum(hs)
        out = out + self.mlp(out)
        out = self.norm3(out)
        
        return out    

class molGraphRepresentation(nn.Module):
    def __init__(self, node_dim, embedding_dim, num_layers,dropout):
        super().__init__()
        self.convs = ModuleList()
        self.node_linear = Sequential(
            Linear(node_dim,embedding_dim),
            ReLU(),
            Linear(embedding_dim,embedding_dim),
        )
        
        for _ in range(num_layers):
            nn = Sequential(
                Linear(embedding_dim, embedding_dim * 2),
                ReLU(),
                Linear(embedding_dim * 2, embedding_dim),
            )
            conv = GPSConv(embedding_dim, GINConv(nn), heads=4, dropout=dropout)
            self.convs.append(conv)

        self.global_fc = Sequential(
            Linear(embedding_dim, 1024),
            ReLU(),
            Dropout(dropout),
            Linear(1024, embedding_dim),
            #Dropout(dropout)
        )
    def forward(self,data):
        data.x = self.node_linear(data.x)
        x, edge_index, batch = data.x, data.edge_index, data.batch
        for conv in self.convs:
            x = conv(x, edge_index, batch)
        x = gep(x,batch)
        x = self.global_fc(x)
        return x

class proGraphRepresentation(nn.Module):
    def __init__(self, num_features_pro,dropout,output_dim):
        super().__init__()
        self.pro_conv = nn.ModuleList([])
        self.pro_conv.append(GCNConv(num_features_pro, num_features_pro * 4))
        self.pro_conv.append(GATConv(num_features_pro * 4, num_features_pro * 4, heads=4, dropout=dropout, concat=False))
        self.pro_conv.append(GATConv(num_features_pro * 4, num_features_pro * 4, heads=4, dropout=dropout, concat=False))
        self.pro_out_feats = num_features_pro * 4
        self.pro_seq_fc1 = nn.Linear(num_features_pro * 4, num_features_pro * 4)
        self.pro_seq_fc2 = nn.Linear(num_features_pro * 4, num_features_pro * 4)
        self.pro_bias = nn.Parameter(torch.rand(1, num_features_pro * 4))
        torch.nn.init.uniform_(self.pro_bias, a=-0.2, b=0.2)
        self.global_fc = Sequential(
            Linear(num_features_pro * 4, 1024),
            ReLU(),
            Dropout(dropout),
            Linear(1024, output_dim),
            Dropout(dropout)
        )
        self.relu = nn.ReLU()
    
    def forward(self,data):
        # get protein input
        x, edge_index, weight, batch = data.x, data.edge_index, data.edge_weight, data.batch
        n = x.size(0)
        for i in range(len(self.pro_conv)):
            if i == 0:
                xc = self.pro_conv[i](x, edge_index, weight)
            else:
                xc = self.pro_conv[i](x, edge_index)
            if i < len(self.pro_conv) - 1:
                xc = self.relu(xc)
            if i == 0:
                x = xc
                continue
            pro_z = torch.sigmoid(
                self.pro_seq_fc1(xc) + self.pro_seq_fc2(x) + self.pro_bias.expand(n, self.pro_out_feats))
            x = pro_z * xc + (1 - pro_z) * x

        x = gmp(x, batch)  # global pooling
        # flatten
        x = self.global_fc(x)
        return x
class Conv1dReLU(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()
        self.inc = nn.Sequential(
            nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,
                      padding=padding),
            nn.ReLU()
        )

    def forward(self, x):
        return self.inc(x)


class LinearReLU(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.inc = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=out_features, bias=bias),
            nn.ReLU()
        )

    def forward(self, x):
        return self.inc(x)


class StackCNN(nn.Module):
    def __init__(self, layer_num, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super().__init__()

        self.inc = nn.Sequential(OrderedDict([('conv_layer0',
                                               Conv1dReLU(in_channels, out_channels, kernel_size=kernel_size,
                                                          stride=stride, padding=padding))]))
        for layer_idx in range(layer_num - 1):
            self.inc.add_module('conv_layer%d' % (layer_idx + 1),
                                Conv1dReLU(out_channels, out_channels, kernel_size=kernel_size, stride=stride,
                                           padding=padding))

        self.inc.add_module('pool_layer', nn.AdaptiveMaxPool1d(1))

    def forward(self, x):
        return self.inc(x).squeeze(-1)


class proSequenceRePresentation(nn.Module):
    def __init__(self, block_num, vocab_size, embedding_num):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_num, padding_idx=0)
        self.block_list = nn.ModuleList()
        for block_idx in range(block_num):
            self.block_list.append(
                StackCNN(block_idx + 1, embedding_num, 96, 3)
            )

        self.linear = nn.Linear(3 * 96, 128)
        
    def forward(self, x):
        x = self.embed(x).permute(0, 2, 1)
        feats = [block(x) for block in self.block_list]

        x = torch.cat(feats, -1)
        x = self.linear(x)

        return x

class MMSGDTA(torch.nn.Module):
    def __init__(self,num_features_pro=33, num_features_mol=88, embed_dim=128, dropout=0.2):
        super(MMSGDTA, self).__init__()

        print('MMSGDTA Loading ...')
        
        self.ligand_encoder = molGraphRepresentation(num_features_mol,num_layers = 4, embedding_dim=embed_dim,dropout=dropout)
        self.progra_encoder = proGraphRepresentation(num_features_pro,dropout,embed_dim)
        self.proseq_encoder = proSequenceRePresentation(block_num=3,vocab_size = 22, embedding_num=128)
        
        self.attention = Attention(embed_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        # combined layers
        self.fc1 = nn.Linear(3 * embed_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.out = nn.Linear(512, 1)

    def forward(self, data_mol, data_pro):
        data_pro_seq = data_pro.pro_emb
        mol_x = self.ligand_encoder(data_mol)
        pro_x = self.progra_encoder(data_pro)
        pro_s = self.proseq_encoder(data_pro_seq)
        
        a = self.attention(mol_x, pro_x, pro_s)
        emb = torch.stack([mol_x, pro_x, pro_s], dim=1)
        a = a.unsqueeze(dim=2)
        fused_emb = (a * emb).reshape(-1, 3 * 128)

        xc = self.fc1(fused_emb)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        return out


In [ ]:
dataset = 'test'
model_file_name = 'Your trained weights file'
device = torch.device("cuda:0")
model = MMSGDTA().to(device)
test_data = create_dataset(dataset)
test_loader = DataLoader(test_data, batch_size= 512, shuffle=False, collate_fn=collate)

MMSGDTA Loading ...
create molecule graph ...


100%|██████████| 1/1 [00:00<00:00, 20.11it/s]


create protein graph ...


100%|██████████| 1/1 [00:00<00:00, 56.99it/s]


loading tensors ...


100%|██████████| 1/1 [00:00<00:00, 1329.41it/s]
/data10/XJH/anaconda3/envs/PCSG/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [80]:
def predicting(model, device,dataloader):
    model.eval()
    total_preds = torch.Tensor()
    #total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(dataloader.dataset)))
    with torch.no_grad():
        for data in dataloader:
            data_mol = data[0].to(device)
            data_pro = data[1].to(device)

            output = model(data_mol, data_pro)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
    
    return total_preds.numpy().flatten()

In [81]:
if os.path.isfile(model_file_name):
    model.load_state_dict(torch.load(model_file_name,map_location=torch.device('cpu')),strict=False)
    P = predicting(model,device,test_loader)

    print(P)

/tmp/ipykernel_305764/910732913.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_file_name,map_location=torch.device('cpu')),strict

Make prediction for 1 samples...
[10.328574]
